In [ ]:
  #!/usr/bin/env python
try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass
from astropy.io import fits
from astropy.table import Table
import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
import sys
sys.path.insert(0,'../../scripts')

from scipy.stats import chi2
import corner
Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from chainconsumer import ChainConsumer , Chain

In [ ]:

def add_mcat(c, mcat, name):
    nwalkers = mcat.nchains ()
    m2lnL = mcat.get_m2lnp_var ()

    # Use nwalkers * 10 as burning length
    burn = nwalkers * burnin
    rows = np.array ([mcat.peek_row (i).dup_array () for i in range (burn, mcat.len ())])
    params = ["$" + mcat.col_symb (i) + "$" for i in range (mcat.ncols ())]

    posterior = -0.5 * rows[:,m2lnL]

    rows   = np.delete (rows,   m2lnL, 1)
    params = np.delete (params, m2lnL, 0)

    indices = [0,1,2,3]

    rows   = rows[:,indices]
    params = params[indices]

    c.add_chain (rows, posterior = posterior, parameters=list(params), name=name, plot_point = True)

In [ ]:
burnin  = 50
area = 2959.1
parameters = [[0.8,4.5, 0.0, 2.560352988400761] , [0.8,4.5,-4.0, 3.139799921784934] , [0.8,2.5,0.0,0.9419012265745043]
              , [0.8,2.5,-4.0,1.15506784061638] , [0.2,4.5,-4.0,2.7559040762338385]  , [0.2,2.5,-4.0,1.0138404514870045]]

burnins_sij_fixed = [[72,48],[192,164],[64 , 48],[48,64],[96,80 ], [128,128] ]
#bayesian_evidence [[-57.4554,-57.2226],[ -50.2294,-50.28589] ,[-54.3498, -54.1503], [ -52.6136,-52.4229], [-58.2653,-57.1903],
#[ -55.8547,-55.9696]]
#1sigma_posterior_volume[[-9.0364,-8.9259],[-4.5679, -4.4616], [ -9.9349, -9.9966] , [ -7.1127, -7.2460],[-8.5115, -8.1756],
#[-10.5865,-10.6072]]



for i in  range(len(parameters)):
    mcat_mcmc = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_0.8.fits" % (parameters[i][2],
                                                                                              parameters[i][0],parameters[i][1] ,
                                                                                              area), burnins_sij_fixed[i][0] * 500)
    
    mcat_mcmc_correct = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_correct_0.8.fits" %
                                                          (parameters[i][2],parameters[i][0],parameters[i][1], area),
                                                          burnins_sij_fixed[i][1] * 500)

    consumer = ChainConsumer()

    add_mcat (consumer, mcat_mcmc_correct, "Correct")   
    add_mcat (consumer, mcat_mcmc, "Incorrect")

    consumer.configure (kde = True, label_font_size=11, cloud=True, sigma2d=True, sigmas = [1, 2, 3], 
                        spacing = 0.0, tick_font_size=11, usetex=False, shade = True)

    fig = consumer.plotter.plot(truth = [parameters[i][3],parameters[i][0], parameters[i][2],parameters[i][1]])

    fig.set_size_inches(12, 12)                                                            
                                     

In [ ]:
#fix cov
area = 2959.1
parameters = [[0.8,4.5, 0.0, 2.560352988400761] , [0.8,4.5,-4.0, 3.139799921784934] , [0.8,2.5,0.0,0.9419012265745043]  ,
              [0.8,2.5,-4.0,1.15506784061638] , [0.2,4.5,-4.0,2.7559040762338385]  , [0.2,2.5,-4.0,1.0138404514870045]]

burnins_cov_fixed = [[64,80],[50, 48],[64,32], [48,64],[192,96], [176,144]]
#bayesian_evidence [[ -78.3986,-57.2561 ],[ -61.5603, -50.1430], [-56.5886,-54.1802] , [-54.1857,-52.4202],[-83.9660,-57.1100 ] , [-63.2439,-55.9984] ]
#1sigma_posterior_volume[[-12.2006, -8.9645 ],[-10.3163, -4.3255], [-12.2189,-10.0222] , [-10.0996, -7.2480], [-11.5463,-8.1178] , [-12.5127,-10.6331]]


for i in  range(len(parameters)):
    mcat_mcmc = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_0.8_fix.fits" % 
                                                          (parameters[i][2],parameters[i][0],parameters[i][1] ,area), burnins_cov_fixed[i][0] * 500)
                                                                            
    mcat_mcmc_correct = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_0.8_fix_correct.fits" % 
                                                          (parameters[i][2],parameters[i][0],parameters[i][1], area), burnins_cov_fixed[i][1] * 500)

    consumer = ChainConsumer()

    add_mcat (consumer, mcat_mcmc_correct, "Correct")   
    add_mcat (consumer, mcat_mcmc, "Incorrect")

    consumer.configure (kde = True, label_font_size=11, cloud=True, sigma2d=True, sigmas = [1, 2, 3], spacing = 0.0, 
                        tick_font_size=11, usetex=False, shade = True)

    fig = consumer.plotter.plot(truth = [parameters[i][3],parameters[i][0], parameters[i][2],parameters[i][1]])

    fig.set_size_inches(12, 12)      

In [ ]:
#fix cov vs fix sij
area = 2959.1
parameters = [[0.8,4.5, 0.0, 2.560352988400761] , [0.8,4.5,-4.0, 3.139799921784934] , [0.8,2.5,0.0,0.9419012265745043]  ,
              [0.8,2.5,-4.0,1.15506784061638] , [0.2,4.5,-4.0,2.7559040762338385]  , [0.2,2.5,-4.0,1.0138404514870045]]

#bayesian_evidence [[ -78.3986,-57.2561 ],[ -61.5603, -50.1430], [-56.5886,-54.1802] , [-54.1857,-52.4202],[-83.9660,-57.1100 ] , [-63.2439,-55.9984] ]
#1sigma_posterior_volume[[-12.2006, -8.9645 ],[-10.3163, -4.3255], [-12.2189,-10.0222] , [-10.0996, -7.2480], [-11.5463,-8.1178] , [-12.5127,-10.6331]]


for i in  range(len(parameters)):
    mcat_mcmc_cov_fixed = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_0.8_fix_correct.fits" % 
                                                          (parameters[i][2],parameters[i][0],parameters[i][1], area), burnins_cov_fixed[i][1] * 500)
                                                                            
    mcat_mcmc_sij_fixed = Ncm.MSetCatalog.new_from_file_ro ("exp_%.2f,%.2f,%.2f_%.1f_correct_0.8.fits" %
                                                          (parameters[i][2],parameters[i][0],parameters[i][1], area),
                                                          burnins_sij_fixed[i][1] * 500)

    consumer = ChainConsumer()

    add_mcat (consumer, mcat_mcmc_correct, "Correct")   
    add_mcat (consumer, mcat_mcmc, "Incorrect")

    consumer.configure (kde = True, label_font_size=11, cloud=True, sigma2d=True, sigmas = [1, 2, 3], spacing = 0.0, 
                        tick_font_size=11, usetex=False, shade = True)

    fig = consumer.plotter.plot(truth = [parameters[i][3],parameters[i][0], parameters[i][2],parameters[i][1]])

    fig.set_size_inches(12, 12) 